In [1]:
# from fastbook import *
from sklearn import datasets
import seaborn as sns
import sklearn
import torch
from torch.utils.data import DataLoader
from torch.nn import functional as F

from fastai.vision.all import DataLoaders, Learner
import fastai

from fastai.vision.all import URLs, cnn_learner, resnet18, accuracy
# from fastbook import URLS
from fastai.vision.all import ImageDataLoaders, untar_data
from torch import nn


from mnist.mean_classifier import MeanClassifier
from mnist import sgd, learner, loss
# from sgd import SGDRegressor, SGDClassifier
from mnist import utils

# Mean Classifier

In [2]:
X_train, X_test, y_train, y_test = utils.load_mnist()
regression_dl = utils.load_linear()

In [3]:
train_dl = DataLoader(list(zip(X_train, y_train)), batch_size=256, shuffle=True)
test_dl = DataLoader(list(zip(X_test, y_test)), batch_size=256)
dls = DataLoaders(train_dl, test_dl)

In [4]:
clf = MeanClassifier(loss='mae')
clf.fit(train_dl)
print(f'Mean Classifier with MAE loss had an accuracy of {clf.score(test_dl):.4}')

Mean Classifier with MAE loss had an accuracy of 0.9514


In [5]:
clf = MeanClassifier(loss='mse')
clf.fit(train_dl)
clf.score(test_dl)
print(f'Mean Classifier with MSE loss had an accuracy of {clf.score(test_dl):.4}')

Mean Classifier with MSE loss had an accuracy of 0.9662


# Gradient Descent

In [6]:
reg = sgd.SGDRegressor(loss=F.mse_loss, epochs=10, lr=1e-3)
reg.fit(regression_dl)
print(f'Gradient Descent Regressor with MSE loss had an R^2 of {reg.score(regression_dl):.4}')

Gradient Descent Regressor with MSE loss had an R^2 of 0.9806


In [7]:
reg = sgd.SGDRegressor(loss=F.l1_loss, epochs=20, lr=1e-2)
reg.fit(regression_dl)
print(f'Gradient Descent Regressor with MAE loss had an R^2 of {reg.score(regression_dl):.4}')

Gradient Descent Regressor with MAE loss had an R^2 of 0.9163


# Classifier

In [8]:
clf = sgd.SGDClassifier(lr=1, epochs=10)
clf.fit(train_dl)
print(f'Gradient Descent Classifier with custom SGD had an accuracy of {clf.score(test_dl):.4}')

Gradient Descent Classifier with custom SGD had an accuracy of 0.98


In [9]:
# trying with pytorch's SGD:
clf = learner.Learner(epochs=10, lr=1, optimiser=torch.optim.SGD, loss=loss.sigmoid_loss, model=torch.nn.Linear(28*28, 1))
clf.fit(train_dl)
preds = clf.predict(X_test)
print(f"Gradient Descent Classifier with torch's SGD had an accuracy of {sgd.accuracy(preds, y_test):.4}")

Gradient Descent Classifier with torch's SGD had an accuracy of 0.9789


In [10]:
learn = Learner(dls, torch.nn.Linear(28*28,1), opt_func=fastai.optimizer.SGD, loss_func=loss.sigmoid_loss, metrics=sgd.accuracy)
learn.fit(10, lr=1)

epoch,train_loss,valid_loss,accuracy,time
0,0.057973,0.041334,0.971050,00:00
1,0.040404,0.034486,0.975957,00:00
2,0.032530,0.031217,0.976938,00:00
3,0.027967,0.029299,0.978410,00:00
4,0.025414,0.028312,0.976938,00:00
5,0.023821,0.027038,0.978901,00:00
6,0.022702,0.025975,0.979392,00:00
7,0.021910,0.025191,0.979392,00:00
8,0.020667,0.025386,0.979392,00:00
9,0.020023,0.024154,0.978901,00:00


# Moving to Neural Networks

In [11]:
simple_net = nn.Sequential(
    nn.Linear(28*28, 30),
    nn.ReLU(),
    nn.Linear(30, 1),
)

In [12]:
clf = sgd.SGDClassifier(1, 10, loss=loss.sigmoid_loss, model=simple_net)
clf.fit(train_dl)
print(f"Neural Network had an accuracy of {clf.score(test_dl):.4}")

Neural Network had an accuracy of 0.979


In [13]:
# and beyond
path = untar_data(URLs.MNIST_SAMPLE)

# dls = ImageDataLoaders.from_folder(path)
learn = cnn_learner(dls, resnet18, pretrained=False,
                    loss_func=F.cross_entropy, metrics=accuracy)


AssertionError: `n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`

In [ ]:
learn.fit_one_cycle(1, 0.1)